In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [35]:
# Imports
import sys
sys.path.append('../')
import time
import string
import json
import os
# General
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from contextlib import redirect_stdout
from pathlib import Path
# brain2brain
from brain2brain import utils
from brain2brain import generators
# TCN
from brain2brain.tcn import TCN
from brain2brain.tcn import compiled_tcn
# TF
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, GRU, Lambda, TimeDistributed, RepeatVector
# Wandb
import wandb
from wandb.keras import WandbCallback

In [29]:
def run_experiment_for_electrode(electrode = 0):
    experiment_dict = {
        "experiment_name": "gru_o2o_sweep_676_bin_norm",
        "experiment_description" : "Attempting to predict signals for all electrodes in order to better understand which ones work. All hyperparameters below get passed to hyperparameters_defaults.",
        "target_folder" : "gru_o2o_sweep_676_bin_norm/",
        "train_path" : "/home/tmarcu/brain2brain/brain2brain/train_676_bin_norm_2.txt",
        "val_path" : "/home/tmarcu/brain2brain/brain2brain/val_676_bin_norm_2.txt",
        "batch_size" : 512,
        "epochs" : 128,
        "early_stopping_patience": 2,
        "lookback_window" : 15,
        "length_pred" : 3,
        "delay_pred" : 0,
        "electrode" : 0,
        "activation" : "relu",
        "samples_per_second" : 1,
        "return_sequences" : True,
        "debug_mode" : False,
        "hidden_units": 100,
        "dropout_rate": 0.05,
        "recurrent_dropout": 0.05,
        "activation" : "linear",
        "opt" : "adam",
        "loss" : "mse"
    }

    experiment_name = experiment_dict['experiment_name']
    experiment_description = experiment_dict['experiment_description']
    target_folder = experiment_dict['target_folder']

    # Ensure target directory exists.
    try:
        Path(target_folder).mkdir(parents=True, exist_ok=True)
    except IOError:
        print(f"Directory creation failed for path {target_folder}")

    # Save dictionary in json as well
    utils.save_json_file(experiment_dict, target_folder + "experiment_params.json")
    # Configure wandb
    # Toggle Offline Mode
    os.environ['WANDB_MODE'] = 'dryrun'
    wandb.init(name=experiment_name,
                notes=experiment_description,
                config=experiment_dict,
                dir=target_folder,
                entity="theodormarcu",
                project="brain2brain")

    # Save Hyperparams
    config = wandb.config
    # Read saved paths for training.
    # saved_paths = utils.get_file_paths(config.path)
    train_paths = utils.get_file_paths(config.train_path)
    val_paths = utils.get_file_paths(config.val_path)
    # Split the train files into a training and validation set.
    # train, val = utils.split_file_paths(saved_paths, 0.8)
    total_electrode_count = utils.get_file_shape(train_paths[0])[1]
    # Electrodes
    electrode_count = 1
    # Sampling of electrodes.
    timesteps_per_sample = int(config.lookback_window // config.samples_per_second)
    # Training Generator
    train_generator = generators.FGenerator(file_paths=train_paths,
                                            lookback=config.lookback_window,
                                            length=config.length_pred,
                                            delay=config.delay_pred,
                                            batch_size=config.batch_size,
                                            sample_period=config.samples_per_second,
                                            electrodes=[config.electrode],
                                            electrode_output_ix=config.electrode,
                                            shuffle=True,
                                            debug=config.debug_mode)
    # Validation Generator
    val_generator = generators.FGenerator(file_paths=val_paths,
                                            lookback=config.lookback_window,
                                            length=config.length_pred,
                                            delay=config.delay_pred,
                                            batch_size=config.batch_size,
                                            sample_period=config.samples_per_second,
                                            electrodes=[config.electrode],
                                            electrode_output_ix=config.electrode,
                                            shuffle=False,
                                            debug=config.debug_mode)

    train_steps = len(train_generator)
    val_steps = len(val_generator)

    print(f"Train Generator Batch Shape:\n"
            f"Sample={train_generator[0][0].shape} Pred={train_generator[0][1].shape}")
    print(f"Validation Generator Batch Shape:\n"
            f"Sample={val_generator[0][0].shape} Pred={val_generator[0][1].shape}")

    callbacks_list = [
        EarlyStopping(
            monitor="val_loss",
            patience=2,
            mode="min"
        ),
        ModelCheckpoint(
            filepath=target_folder+"model_checkpoint.h5",
            monitor="val_loss",
            save_best_only=True,
        ),
        WandbCallback(
            monitor="val_loss",
            mode="min",
            save_model=True,
        )
    ]
    # GRU One-to-one Architecture
    model = Sequential()
    model.add(GRU(config.hidden_units,
                  activation=config.activation,
                  return_sequences=config.return_sequences,
                  input_shape=(config.lookback_window, 1),
                  dropout=config.dropout_rate,
                  recurrent_dropout=config.recurrent_dropout))
    print(model.output_shape)
    model.add(GRU(config.hidden_units, activation=config.activation))
    model.add(Dense(config.length_pred))
    model.compile(optimizer=config.opt, loss=config.loss)
    model.summary()

    # Save Model Config and Architecture
    utils.save_json_file(model.get_config(), target_folder + "model_config.json")
    utils.save_json_file(model.to_json(), target_folder + "model_architecture.json")

    history = model.fit_generator(generator=train_generator,
                                    steps_per_epoch=train_steps,
                                    epochs=config.epochs,
                                    callbacks=callbacks_list,
                                    validation_data=val_generator,
                                    validation_steps=val_steps)

    model.save(target_folder + "model.h5")
    model.save_weights(target_folder + 'model_weights.h5')
    # Save History to File (For Later)
    utils.save_json_file(history.history, target_folder + "history.json")
    # Plot Loss Curves for Validation and Training
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.figure()
    plt.plot(epochs, loss, 'bo', label="Training Loss")
    plt.plot(epochs, val_loss, 'b', label="Validation Loss")
    plt.title(f"Training and Validation Loss. Electrode {config.electrode}")
    plt.savefig(target_folder + "train_val_loss_plot.png")
    plt.clf()

In [30]:
run_experiment_for_electrode(0)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_28 (GRU)                 (None, 15, 100)           30900     
_________________________________________________________________
gru_29 (GRU)                 (None, 100)               60600     
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 31s 177ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [========

<Figure size 432x288 with 0 Axes>

In [8]:
run_experiment_for_electrode(1)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 15, 100)           30900     
_________________________________________________________________
gru_3 (GRU)                  (None, 100)               60600     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 33s 188ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [=========

<Figure size 432x288 with 0 Axes>

In [9]:
run_experiment_for_electrode(2)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 15, 100)           30900     
_________________________________________________________________
gru_5 (GRU)                  (None, 100)               60600     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 30s 175ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [=========

<Figure size 432x288 with 0 Axes>

In [10]:
run_experiment_for_electrode(3)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_6 (GRU)                  (None, 15, 100)           30900     
_________________________________________________________________
gru_7 (GRU)                  (None, 100)               60600     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 31s 181ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [=========

<Figure size 432x288 with 0 Axes>

In [11]:
run_experiment_for_electrode(4)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_8 (GRU)                  (None, 15, 100)           30900     
_________________________________________________________________
gru_9 (GRU)                  (None, 100)               60600     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 31s 181ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [=========

<Figure size 432x288 with 0 Axes>

In [12]:
run_experiment_for_electrode(5)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_10 (GRU)                 (None, 15, 100)           30900     
_________________________________________________________________
gru_11 (GRU)                 (None, 100)               60600     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 32s 185ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [=========

<Figure size 432x288 with 0 Axes>

In [13]:
run_experiment_for_electrode(45)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_12 (GRU)                 (None, 15, 100)           30900     
_________________________________________________________________
gru_13 (GRU)                 (None, 100)               60600     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 32s 185ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [=========

<Figure size 432x288 with 0 Axes>

In [23]:
run_experiment_for_electrode(75)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_22 (GRU)                 (None, 15, 100)           30900     
_________________________________________________________________
gru_23 (GRU)                 (None, 100)               60600     
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 31s 177ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [========

<Figure size 432x288 with 0 Axes>

In [24]:
run_experiment_for_electrode(100)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3)
(None, 15, 100)
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_24 (GRU)                 (None, 15, 100)           30900     
_________________________________________________________________
gru_25 (GRU)                 (None, 100)               60600     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 303       
Total params: 91,803
Trainable params: 91,803
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epoch 1/128
173/173 [==============================] - 31s 179ms/step - loss: nan - val_loss: nan
Epoch 2/128
173/173 [========

<Figure size 432x288 with 0 Axes>

In [38]:
def run_experiment_for_electrode_seq2seq(electrode = 0):
    experiment_dict = {
        "experiment_name": "gru_o2o_sweep_676_bin_norm_seq2seq",
        "experiment_description" : "Attempting to predict signals for all electrodes in order to better understand which ones work. All hyperparameters below get passed to hyperparameters_defaults.",
        "target_folder" : "gru_o2o_sweep_676_bin_norm_seq2seq/",
        "train_path" : "/home/tmarcu/brain2brain/brain2brain/train_676_bin_norm_2.txt",
        "val_path" : "/home/tmarcu/brain2brain/brain2brain/val_676_bin_norm_2.txt",
        "batch_size" : 512,
        "epochs" : 128,
        "early_stopping_patience": 2,
        "lookback_window" : 15,
        "length_pred" : 3,
        "delay_pred" : 0,
        "electrode" : 0,
        "activation" : "relu",
        "samples_per_second" : 1,
        "return_sequences" : True,
        "debug_mode" : False,
        "hidden_units": 100,
        "dropout_rate": 0.05,
        "recurrent_dropout": 0.05,
        "activation" : "linear",
        "opt" : "adam",
        "loss" : "mse"
    }

    experiment_name = experiment_dict['experiment_name']
    experiment_description = experiment_dict['experiment_description']
    target_folder = experiment_dict['target_folder']

    # Ensure target directory exists.
    try:
        Path(target_folder).mkdir(parents=True, exist_ok=True)
    except IOError:
        print(f"Directory creation failed for path {target_folder}")

    # Save dictionary in json as well
    utils.save_json_file(experiment_dict, target_folder + "experiment_params.json")
    # Configure wandb
    # Toggle Offline Mode
    os.environ['WANDB_MODE'] = 'dryrun'
    wandb.init(name=experiment_name,
                notes=experiment_description,
                config=experiment_dict,
                dir=target_folder,
                entity="theodormarcu",
                project="brain2brain")

    # Save Hyperparams
    config = wandb.config
    # Read saved paths for training.
    # saved_paths = utils.get_file_paths(config.path)
    train_paths = utils.get_file_paths(config.train_path)
    val_paths = utils.get_file_paths(config.val_path)
    # Split the train files into a training and validation set.
    # train, val = utils.split_file_paths(saved_paths, 0.8)
    total_electrode_count = utils.get_file_shape(train_paths[0])[1]
    # Electrodes
    electrode_count = 1
    # Sampling of electrodes.
    timesteps_per_sample = int(config.lookback_window // config.samples_per_second)
    # Training Generator
    train_generator = generators.FGenerator(file_paths=train_paths,
                                            lookback=config.lookback_window,
                                            length=config.length_pred,
                                            delay=config.delay_pred,
                                            batch_size=config.batch_size,
                                            sample_period=config.samples_per_second,
                                            electrodes=[config.electrode],
                                            electrode_output_ix=config.electrode,
                                            shuffle=True,
                                            debug=config.debug_mode)
    # Validation Generator
    val_generator = generators.FGenerator(file_paths=val_paths,
                                            lookback=config.lookback_window,
                                            length=config.length_pred,
                                            delay=config.delay_pred,
                                            batch_size=config.batch_size,
                                            sample_period=config.samples_per_second,
                                            electrodes=[config.electrode],
                                            electrode_output_ix=config.electrode,
                                            shuffle=False,
                                            debug=config.debug_mode)

    train_steps = len(train_generator)
    val_steps = len(val_generator)

    print(f"Train Generator Batch Shape:\n"
            f"Sample={train_generator[0][0].shape} Pred={train_generator[0][1].shape}")
    print(f"Validation Generator Batch Shape:\n"
            f"Sample={val_generator[0][0].shape} Pred={val_generator[0][1].shape}")

    callbacks_list = [
        EarlyStopping(
            monitor="val_loss",
            patience=2,
            mode="min"
        ),
        ModelCheckpoint(
            filepath=target_folder+"model_checkpoint.h5",
            monitor="val_loss",
            save_best_only=True,
        ),
        WandbCallback(
            monitor="val_loss",
            mode="min",
            save_model=True,
        )
    ]
    # GRU One-to-one Architecture
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(config.lookback_window, 1)))
    model.add(RepeatVector(3))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    model.summary()

    # Save Model Config and Architecture
    utils.save_json_file(model.get_config(), target_folder + "model_config.json")
    utils.save_json_file(model.to_json(), target_folder + "model_architecture.json")

    history = model.fit_generator(generator=train_generator,
                                    steps_per_epoch=train_steps,
                                    epochs=config.epochs,
                                    callbacks=callbacks_list,
                                    validation_data=val_generator,
                                    validation_steps=val_steps)

    model.save(target_folder + "model.h5")
    model.save_weights(target_folder + 'model_weights.h5')
    # Save History to File (For Later)
    utils.save_json_file(history.history, target_folder + "history.json")
    # Plot Loss Curves for Validation and Training
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.figure()
    plt.plot(epochs, loss, 'bo', label="Training Loss")
    plt.plot(epochs, val_loss, 'b', label="Validation Loss")
    plt.title(f"Training and Validation Loss. Electrode {config.electrode}")
    plt.savefig(target_folder + "train_val_loss_plot.png")
    plt.clf()

In [41]:
run_experiment_for_electrode_seq2seq(0)

(5429, 114)


TypeError: data type not understood

In [42]:
run_experiment_for_electrode_seq2seq(1)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3, 1)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3, 1)
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 200)               161600    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 3, 200)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 3, 200)            320800    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 3, 1)              201       
Total params: 482,601
Trainable params: 482,601
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epo

<Figure size 432x288 with 0 Axes>

In [43]:
run_experiment_for_electrode_seq2seq(16)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3, 1)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3, 1)
Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 200)               161600    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 3, 200)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 3, 200)            320800    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 3, 1)              201       
Total params: 482,601
Trainable params: 482,601
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epo

<Figure size 432x288 with 0 Axes>

In [44]:
run_experiment_for_electrode_seq2seq(10)

(5429, 114)
Train Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3, 1)
Validation Generator Batch Shape:
Sample=(512, 15, 1) Pred=(512, 3, 1)
Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 200)               161600    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 3, 200)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 3, 200)            320800    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 3, 1)              201       
Total params: 482,601
Trainable params: 482,601
Non-trainable params: 0
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 173 steps, validate for 57 steps
Epo

<Figure size 432x288 with 0 Axes>

In [ ]:
2